In [1]:
import numpy as np 
import pandas as pd 
import gc

pd.options.mode.chained_assignment = None  # default='warn'
#import matplotlib.pyplot as plt
#import seaborn as sns
df_cal = pd.read_csv('calendar.csv')
df_eval = pd.read_csv('sales_train_evaluation.csv')
df_price = pd.read_csv('sell_prices.csv')
#df_sample_output = pd.read_csv('sample_submission.csv')

In [2]:
df_cal

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1964,2016-06-15,11620,Wednesday,5,6,2016,d_1965,NaN,NaN,NaN,NaN,0,1,1
1965,2016-06-16,11620,Thursday,6,6,2016,d_1966,NaN,NaN,NaN,NaN,0,0,0
1966,2016-06-17,11620,Friday,7,6,2016,d_1967,NaN,NaN,NaN,NaN,0,0,0
1967,2016-06-18,11621,Saturday,1,6,2016,d_1968,NaN,NaN,NaN,NaN,0,0,0


In [3]:
hol_list = df_cal['event_name_1'].unique()[1:]
wkd_list = ['Saturday','Sunday']
df_cal['is_holiday_1'] = df_cal['event_name_1'].apply(lambda x : 1 if x in hol_list else 0 )
df_cal['is_holiday_2'] = df_cal['event_name_1'].apply(lambda x : 1 if x in hol_list else 0 )
df_cal['is_holiday'] = df_cal[['is_holiday_1','is_holiday_2']].max(axis=1)
df_cal['is_weekend'] = df_cal['weekday'].apply(lambda x : 1 if x in wkd_list else 0 )
df_cal = df_cal.drop(['weekday', 'wday', 'month', 'year', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2'], axis='columns')

In [4]:
df_cal

,date,wm_yr_wk,d,snap_CA,snap_TX,snap_WI,is_holiday_1,is_holiday_2,is_holiday,is_weekend
0,2011-01-29,11101,d_1,0,0,0,0,0,0,1
1,2011-01-30,11101,d_2,0,0,0,0,0,0,1
2,2011-01-31,11101,d_3,0,0,0,0,0,0,0
3,2011-02-01,11101,d_4,1,1,0,0,0,0,0
4,2011-02-02,11101,d_5,1,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
1964,2016-06-15,11620,d_1965,0,1,1,0,0,0,0
1965,2016-06-16,11620,d_1966,0,0,0,0,0,0,0
1966,2016-06-17,11620,d_1967,0,0,0,0,0,0,0
1967,2016-06-18,11621,d_1968,0,0,0,0,0,0,1


In [5]:
df_eval

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,4,0,0,0,0,3,3,0,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,2,1,1,0,0,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,2,0,0,0,2,3,0,1
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,1,0,4,0,1,3,0,2,6
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,2,1,0,0,2,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_evaluation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,0,2,2,...,1,0,3,0,1,1,0,0,1,1
30486,FOODS_3_824_WI_3_evaluation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
30487,FOODS_3_825_WI_3_evaluation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0,6,0,2,...,0,0,1,2,0,1,0,1,0,2
30488,FOODS_3_826_WI_3_evaluation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,1,1,1,4,6,0,1,1,1,0


In [6]:
# for all FOODS_1
eva_all = df_eval[df_eval['dept_id'] == 'FOODS_1']
eva_all = eva_all.melt(['id','item_id','dept_id','cat_id','store_id','state_id'],var_name = 'd', value_name = 'sales')
eva_all = pd.merge(eva_all, df_cal, how = 'left', on = 'd')
eva_all = pd.merge(eva_all, df_price, how = 'left', on = ['item_id', 'wm_yr_wk', 'store_id'])
eva_all = eva_all.drop(['is_holiday_1','is_holiday_2'], axis =1)
eva_all
############################################################ Times cost: 107 seconds

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,date,wm_yr_wk,snap_CA,snap_TX,snap_WI,is_holiday,is_weekend,sell_price
0,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_1,3,2011-01-29,11101,0,0,0,0,1,2.00
1,FOODS_1_002_CA_1_evaluation,FOODS_1_002,FOODS_1,FOODS,CA_1,CA,d_1,0,2011-01-29,11101,0,0,0,0,1,7.88
2,FOODS_1_003_CA_1_evaluation,FOODS_1_003,FOODS_1,FOODS,CA_1,CA,d_1,0,2011-01-29,11101,0,0,0,0,1,2.88
3,FOODS_1_004_CA_1_evaluation,FOODS_1_004,FOODS_1,FOODS,CA_1,CA,d_1,0,2011-01-29,11101,0,0,0,0,1,NaN
4,FOODS_1_005_CA_1_evaluation,FOODS_1_005,FOODS_1,FOODS,CA_1,CA,d_1,3,2011-01-29,11101,0,0,0,0,1,2.94
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4192555,FOODS_1_215_WI_3_evaluation,FOODS_1_215,FOODS_1,FOODS,WI_3,WI,d_1941,0,2016-05-22,11617,0,0,0,0,1,1.84
4192556,FOODS_1_216_WI_3_evaluation,FOODS_1_216,FOODS_1,FOODS,WI_3,WI,d_1941,2,2016-05-22,11617,0,0,0,0,1,5.28
4192557,FOODS_1_217_WI_3_evaluation,FOODS_1_217,FOODS_1,FOODS,WI_3,WI,d_1941,1,2016-05-22,11617,0,0,0,0,1,3.54
4192558,FOODS_1_218_WI_3_evaluation,FOODS_1_218,FOODS_1,FOODS,WI_3,WI,d_1941,20,2016-05-22,11617,0,0,0,0,1,0.98


In [7]:
len(eva_all['item_id'].unique())

216

In [8]:
cat_encode = {"HOBBIES":1,"HOUSEHOLD":2,"FOODS":3}
state_encode = {"CA":1,"TX":2,"WI":3}
dept_encode = {'HOBBIES_1':11, 'HOBBIES_2':12, 
               'HOUSEHOLD_1':21, 'HOUSEHOLD_2':22, 
               'FOODS_1':31,'FOODS_2':32, 'FOODS_3':33}
store_encode = {'CA_1':11, 'CA_2':12, 'CA_3':13, 'CA_4':14, 
                'TX_1':21, 'TX_2':22, 'TX_3':23, 
                'WI_1':31, 'WI_2':32, 'WI_3':33}
def data_encode(df):
    #other_columns = ['cat_id','state_id']
    # use only numerical columns
    df['state_id'] = df['state_id'].replace(state_encode)
    df['cat_id'] = df["cat_id"].replace(cat_encode)
    df['dept_id'] = df["dept_id"].replace(dept_encode)
    df['store_id'] = df["store_id"].replace(store_encode)
    import re
    df['d'] = df['d'].apply(lambda x : int(re.search(r'd_(\d+)', x).group(1)))
    # calendar [["wm_yr_wk", "wday", "month", "year",'d','snap_CA','snap_TX',"snap_WI"]]
    return df

def trans_str(ori_str):
    trans_str = ori_str.replace('_', '')
    trans_str = trans_str.replace('HOBBIES', '1')
    trans_str = trans_str.replace('HOUSEHOLD', '2')
    trans_str = trans_str.replace('FOODS', '3')
    trans_str = trans_str.replace('CA', '1')
    trans_str = trans_str.replace('TX', '2')
    trans_str = trans_str.replace('WI', '3')
    trans_str = trans_str.replace('evaluation', '')
    return trans_str
eva_all_encode = data_encode(eva_all)
eva_all_encode['item_id'] = eva_all_encode['item_id'].apply(trans_str)
eva_all_encode['item_id'] = eva_all_encode['item_id'].astype(int)
eva_all_encode['id'] = eva_all_encode['id'].apply(trans_str)
eva_all_encode['id'] = eva_all_encode['id'].astype(int)
uni_val = eva_all_encode['id'].unique()
mapping = {val: idx for idx, val in enumerate(uni_val)}
eva_all_encode['id'] = np.vectorize(mapping.get)(eva_all_encode['id'])
eva_all_encode = eva_all_encode.drop(['date','wm_yr_wk'],axis=1)#,'id'

eva_all_encode
############################################## Time cost : 121 seconds

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,snap_CA,snap_TX,snap_WI,is_holiday,is_weekend,sell_price
0,0,31001,31,3,11,1,1,3,0,0,0,0,1,2.00
1,1,31002,31,3,11,1,1,0,0,0,0,0,1,7.88
2,2,31003,31,3,11,1,1,0,0,0,0,0,1,2.88
3,3,31004,31,3,11,1,1,0,0,0,0,0,1,NaN
4,4,31005,31,3,11,1,1,3,0,0,0,0,1,2.94
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4192555,2155,31215,31,3,33,3,1941,0,0,0,0,0,1,1.84
4192556,2156,31216,31,3,33,3,1941,2,0,0,0,0,1,5.28
4192557,2157,31217,31,3,33,3,1941,1,0,0,0,0,1,3.54
4192558,2158,31218,31,3,33,3,1941,20,0,0,0,0,1,0.98


In [9]:
len(eva_all['id'].unique())

2160

In [10]:
eva_all_encode.isna().sum()

id                 0
item_id            0
dept_id            0
cat_id             0
store_id           0
state_id           0
d                  0
sales              0
snap_CA            0
snap_TX            0
snap_WI            0
is_holiday         0
is_weekend         0
sell_price    601342
dtype: int64

In [11]:
eva_f1 = eva_all_encode#.drop('id',axis=1)
eva_f1

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,snap_CA,snap_TX,snap_WI,is_holiday,is_weekend,sell_price
0,0,31001,31,3,11,1,1,3,0,0,0,0,1,2.00
1,1,31002,31,3,11,1,1,0,0,0,0,0,1,7.88
2,2,31003,31,3,11,1,1,0,0,0,0,0,1,2.88
3,3,31004,31,3,11,1,1,0,0,0,0,0,1,NaN
4,4,31005,31,3,11,1,1,3,0,0,0,0,1,2.94
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4192555,2155,31215,31,3,33,3,1941,0,0,0,0,0,1,1.84
4192556,2156,31216,31,3,33,3,1941,2,0,0,0,0,1,5.28
4192557,2157,31217,31,3,33,3,1941,1,0,0,0,0,1,3.54
4192558,2158,31218,31,3,33,3,1941,20,0,0,0,0,1,0.98


In [14]:
del eva_all_encode
gc.collect()

0

In [15]:
# Level data generation:
col_1_L1 = ['sell_price','sales', 'item_id','store_id', 'state_id'] #,'dept_id', 'cat_id' 'id',
col_1_L2 = ['sell_price','sales', 'item_id','store_id'] #,'dept_id', 'cat_id', 'state_id' 'id',
col_1_L3 = ['sell_price','sales', 'item_id'] #,'dept_id', 'cat_id', 'store_id'            'id', 
col_1_L4 = ['sell_price','sales', ] #'id',

col_2_L1 = ['is_weekend','is_holiday','snap_CA','snap_TX','snap_WI']
col_2_L2 = ['is_weekend','is_holiday','snap_CA','snap_TX','snap_WI']
col_2_L3 = ['is_weekend','is_holiday','snap_CA','snap_TX','snap_WI']
col_2_L4 = ['is_weekend','is_holiday','snap_CA','snap_TX','snap_WI']

col_1_L = [col_1_L1, 
           col_1_L2,
           col_1_L3,
           col_1_L4]
col_2_L = [col_2_L1, 
           col_2_L2,
           col_2_L3,
           col_2_L4]
columns_name = ['d','item_id', 'store_id', 'state_id', 'snap_CA', 'snap_TX', 'snap_WI', 'is_holiday','is_weekend', 'sell_price','sales']#'dept_id', 'cat_id','id',

In [16]:
# generrate the hierarchical time series based on the 4 levels
def levelG(df,col_1, col_2, level_n, columns):
    df_list = []
    col = []
    for i in [0,1,2,3]:
        col = list(set(col_1[0]) - set(col_1[i]))
        col.append('d')
        sum_L = df.groupby(col)[col_1[i]].sum().reset_index()
        mea_L = df.groupby(col)[col_2[i]].mean().reset_index().iloc[:,i+1:]
        level = pd.concat([sum_L,mea_L],axis=1)[columns]
        df_list.append(level)
    return df_list

In [17]:
eva_list = levelG(eva_f1, col_1_L, col_2_L, 4, columns_name)

In [18]:
eva_list[0]

,d,item_id,store_id,state_id,snap_CA,snap_TX,snap_WI,is_holiday,is_weekend,sell_price,sales
0,1,67198180,45792,4104,0.0,0.0,0.0,0.0,1.0,2988.53,2343
1,2,67198180,45792,4104,0.0,0.0,0.0,0.0,1.0,2988.53,2216
2,3,67198180,45792,4104,0.0,0.0,0.0,0.0,0.0,2988.53,1657
3,4,67198180,45792,4104,1.0,1.0,0.0,0.0,0.0,2988.53,1508
4,5,67198180,45792,4104,1.0,0.0,1.0,0.0,0.0,2988.53,1209
...,...,...,...,...,...,...,...,...,...,...,...
1936,1937,67198180,45792,4104,0.0,0.0,0.0,0.0,0.0,7367.44,3073
1937,1938,67198180,45792,4104,0.0,0.0,0.0,0.0,0.0,7367.44,3251
1938,1939,67198180,45792,4104,0.0,0.0,0.0,0.0,0.0,7367.44,3735
1939,1940,67198180,45792,4104,0.0,0.0,0.0,0.0,1.0,7368.77,4195


In [19]:
eva_list[1]

,d,item_id,store_id,state_id,snap_CA,snap_TX,snap_WI,is_holiday,is_weekend,sell_price,sales
0,1,26879272,10800,1,0.0,0.0,0.0,0.0,1.0,1198.90,1157
1,2,26879272,10800,1,0.0,0.0,0.0,0.0,1.0,1198.90,1142
2,3,26879272,10800,1,0.0,0.0,0.0,0.0,0.0,1198.90,787
3,4,26879272,10800,1,1.0,1.0,0.0,0.0,0.0,1198.90,743
4,5,26879272,10800,1,1.0,0.0,1.0,0.0,0.0,1198.90,729
...,...,...,...,...,...,...,...,...,...,...,...
5818,1937,20159454,20736,3,0.0,0.0,0.0,0.0,0.0,2280.30,950
5819,1938,20159454,20736,3,0.0,0.0,0.0,0.0,0.0,2280.30,1048
5820,1939,20159454,20736,3,0.0,0.0,0.0,0.0,0.0,2280.30,1106
5821,1940,20159454,20736,3,0.0,0.0,0.0,0.0,1.0,2280.68,1120


In [20]:
eva_list[3]########  FOODS_1_219_WI_3 : 3_1_219_3_3

,d,item_id,store_id,state_id,snap_CA,snap_TX,snap_WI,is_holiday,is_weekend,sell_price,sales
0,1,31001,11,1,0.0,0.0,0.0,0.0,1.0,2.00,3
1,2,31001,11,1,0.0,0.0,0.0,0.0,1.0,2.00,0
2,3,31001,11,1,0.0,0.0,0.0,0.0,0.0,2.00,0
3,4,31001,11,1,1.0,1.0,0.0,0.0,0.0,2.00,1
4,5,31001,11,1,1.0,0.0,1.0,0.0,0.0,2.00,4
...,...,...,...,...,...,...,...,...,...,...,...
4192555,1937,31219,33,3,0.0,0.0,0.0,0.0,0.0,2.24,0
4192556,1938,31219,33,3,0.0,0.0,0.0,0.0,0.0,2.24,0
4192557,1939,31219,33,3,0.0,0.0,0.0,0.0,0.0,2.24,0
4192558,1940,31219,33,3,0.0,0.0,0.0,0.0,1.0,2.24,0


In [22]:
eva_list[3]['id'].unique()#total 216, absent : 31007, 31100, and  

array([   0,    1,    2, ..., 2157, 2158, 2159])

In [27]:
#### collect_xy for TOP LEVEL (L1)
def collect_xy(df,colname_1,colname_2,x_window:int = 60, y_window: int = 28):
    # for generating x
    df_ = df.copy()
    for i in range(x_window):
        df_[f'x_{i+1}DaysAgo'] = df_[colname_1].shift(i+1)
        
    # for generating y
    for i in range(y_window):
        df_[f"y_{i+1}DaysF"] = df_[colname_2].shift(-i-1)
    df_ = df_.iloc[x_window : -y_window,:]
    return df_

### cominbe_df_xy for other levels
'''
def combine_df_xy(df,colname_3, col_s):
    from tqdm import tqdm
    uni_id = df[colname_3].unique()
    df_c = pd.DataFrame()
    for i in tqdm(range(len(uni_id))):
        df_e = df[df[colname_3]==uni_id[i]]
        df_d = collect_xy(df_e ,col_s ,col_s)
        df_c = pd.concat([df_c,df_d])
    return df_c'''

'\ndef combine_df_xy(df,colname_3, col_s):\n    from tqdm import tqdm\n    uni_id = df[colname_3].unique()\n    df_c = pd.DataFrame()\n    for i in tqdm(range(len(uni_id))):\n        df_e = df[df[colname_3]==uni_id[i]]\n        df_d = collect_xy(df_e ,col_s ,col_s)\n        df_c = pd.concat([df_c,df_d])\n    return df_c'

In [42]:
### cominbe_df_xy for other levels
def combine_df_xy(df,group_n, col_list):
    from tqdm import tqdm
    uni_id = df[col_list[0]].unique()
    df_c = pd.DataFrame()
    if group_n == 1:
        for i in tqdm(range(len(uni_id))):
            df_e = df[df[col_list[0]]==uni_id[i]]
            df_e = collect_xy(df_e ,col_list[-1] ,col_list[-1])
            df_c = pd.concat([df_c,df_e])
    elif group_n == 2 : # This number is the amount of level names required for us to disaggregate the hierarchical structure
        for i in tqdm(range(len(uni_id))):
            uni_id_2 = df[col_list[1]].unique()
            for x in range(len(uni_id_2)):
                      df_e = df[(df[col_list[0]]==uni_id[i]) & (df[col_list[1]]==uni_id_2[x])]
                      df_e = collect_xy(df_e ,col_list[-1] ,col_list[-1])
                      df_c = pd.concat([df_c,df_e])
    return df_c

In [ ]:
l1 = collect_xy(eva_list[0],'sales','sales')
l2 = combine_df_xy(eva_list[1],1,['state_id', 'sales'])
l3 = combine_df_xy(eva_list[2],1,['store_id', 'sales'])
l4 = combine_df_xy(eva_list[3],2,['store_id','item_id','sales'])
#40S

 70%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                                                          | 7/10 [06:55<03:57, 79.26s/it]

In [26]:
eva_list[3][(eva_list[3]['store_id']==11) & (eva_list[3]['item_id']==31002)]

,d,item_id,store_id,state_id,snap_CA,snap_TX,snap_WI,is_holiday,is_weekend,sell_price,sales
1941,1,31002,11,1,0.0,0.0,0.0,0.0,1.0,7.88,0
1942,2,31002,11,1,0.0,0.0,0.0,0.0,1.0,7.88,1
1943,3,31002,11,1,0.0,0.0,0.0,0.0,0.0,7.88,0
1944,4,31002,11,1,1.0,1.0,0.0,0.0,0.0,7.88,1
1945,5,31002,11,1,1.0,0.0,1.0,0.0,0.0,7.88,1
...,...,...,...,...,...,...,...,...,...,...,...
3877,1937,31002,11,1,0.0,0.0,0.0,0.0,0.0,9.48,1
3878,1938,31002,11,1,0.0,0.0,0.0,0.0,0.0,9.48,0
3879,1939,31002,11,1,0.0,0.0,0.0,0.0,0.0,9.48,1
3880,1940,31002,11,1,0.0,0.0,0.0,0.0,1.0,9.48,1


In [41]:
l4

,d,item_id,store_id,state_id,snap_CA,snap_TX,snap_WI,is_holiday,is_weekend,sell_price,...,y_19DaysF,y_20DaysF,y_21DaysF,y_22DaysF,y_23DaysF,y_24DaysF,y_25DaysF,y_26DaysF,y_27DaysF,y_28DaysF


In [27]:
def dataSplit(df):
    x_train, y_train = df[ df['d']<1913 ].iloc[:,:-28], df[df['d']<1913].iloc[:,-28:]
    x_test, y_test   = df[df['d'] >= 1913].iloc[:,:-28], df[df['d'] >= 1913].iloc[:,-28:]
    return x_train,x_test,y_train,y_test

def Concat(df1,df2):
    df = pd.concat([df1,df2], axis=0)
    return df

In [39]:
del x_train, x_test, y_train, y_test

In [40]:
hts = [l1, l2, l3, l4]
x_train, x_test, y_train, y_test = pd.DataFrame(),pd.DataFrame(),pd.DataFrame(),pd.DataFrame()
for i in range(len(hts)) :
    a, b, c, d = dataSplit(hts[i])
x_train, x_test, y_train, y_test = Concat(x_train, a), Concat(x_test, b), Concat(y_train, c), Concat(y_test, d)

TypeError: 'RegressorChain' object is not subscriptable

In [41]:
y_train

""


In [ ]:
y_test

In [ ]:
from sklearn.multioutput import RegressorChain
from sklearn.metrics import mean_squared_error as mse
def get_lgb():
    from lightgbm import LGBMRegressor as lgbr
    return lgbr(
        n_jobs = -1,
        boosting_type= 'gbdt',
        objective = "regression",
        metric = "rmse",
        num_leaves =64,
        learning_rate = 0.1,
        # 0.05 - 74~75
        bagging_fraction = 0.7,
        feature_fraction = 0.5,
        bagging_frequency = 5,
        bagging_seed = 42,
        verbosity = 1,
        seed= 42,
        force_col_wise=True)
#for model_name, get_model in [("LightGB",get_lgb)]:
    #print(f"Model: {model_name}")
l1 = RegressorChain(get_lgb())
l1.fit(x_train, y_train)

print(f"Train score: {mse(multi_target_model.predict(x_train), y_train, squared=True)}")
print(f"Test score: {mse(multi_target_model.predict(x_test), y_test, squared=True)}")

In [45]:
from sklearn.metrics import mean_squared_error as mse
train_pred = l1.predict(x_train)
test_pred = l1.predict(x_test)
print(f"Train score: {mse(train_pred, y_train, squared=True)}")
print(f"Test score: {mse(test_pred, y_test, squared=True)}")

[LightGBM] [Warning] Unknown parameter: bagging_frequency
[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Warning] Unknown parameter: bagging_frequency
[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
Train score: 28.600073255083256
Test score: 6.859432660631401


,y_28DaysF
1912,4031.0
1912,2041.0
3853,962.0
5794,1028.0
1912,334.0
...,...
4190614,0.0
4190615,0.0
4190616,0.0
4190617,0.0


In [44]:
def forecast_mase(forecast, truth):
    mase = []
    from sklearn.metrics import mean_absolute_error as mae
    for i in range(int(len(forecast)/28)):
        mase.append(mae(forecast[i*28:(i+1)*28], truth[i*28:(i+1)*28])/(truth[0:28].diff().abs()[1:].mean()).iloc[0])
    return mase

In [332]:
mase_base = forecast_mase(test_pred, y_test)
mase_base

[1.2370923333972523,
 0.3495827239593794,
 0.24571446994381882,
 0.33801522545317625,
 0.10831853628641508,
 0.18129631133336985,
 0.18545207343491052,
 0.08884575279920874,
 0.09719255400860616,
 0.0954754965885576,
 0.13361800802394086,
 0.10564499804419401,
 0.16679244615687697,
 0.11224970679138298,
 0.0029360062739662053,
 0.002612659172393415,
 0.0028334428800828536,
 0.001192991105280038,
 0.0020009061433428864,
 0.001243371117635528,
 0.001930809617333698,
 0.0013406659117663267,
 0.002272185600984428,
 0.0012422634760578535,
 0.0017282705212448696,
 0.0020880944453854204,
 0.002139447411939254,
 0.0012673301498940263,
 0.001674652805732887,
 0.0013255970616400058,
 0.0012981651020675569,
 0.002415356108121628,
 0.0014358250009507,
 0.0010058437163143195,
 0.0023505416866113865,
 0.003919328576000563,
 0.0029400800019888968,
 0.0013772544535800638,
 0.001145369227233319,
 0.002499033729424064,
 0.0005314628774199466,
 0.0027538464243895204,
 0.001655109988459716,
 0.00180719188

In [32]:
len(test_pred)/28

2174.0

In [333]:
#### mase for the hierarchical time series
np.mean(mase_base)

0.00492575781142385

In [47]:
len(eva_list[3]['store_id'].unique())

10

In [48]:
################################# SUMMING MATRIX FOR BU, TD, AND MINT
def SMatrix(level_number, bottom_level_number, eva_list_ts):
    l = []
    i = 0
    while i <= level_number:
        if i == 0:  # （1，2160）
            # for top level
            a = np.ones(bottom_level_number).reshape(1,2160)
            l.append(a)
            
        elif i == 1:  #（3，2160）
            # for l2
            l.append([])
            o = [[1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
                 [0, 0, 0, 0, 1, 1, 1, 0, 0, 0],
                 [0, 0, 0, 0, 0, 0, 0, 1, 1, 1]]
            for x1 in range(len(eva_list_ts[i]['state_id'].unique())):
                r = (o[x1] * (bottom_level_number // len(o) + 1))[:bottom_level_number]
                l[i].append(r)
        elif i == 2:  #（10，2160）
            # for l3 store
            l.append([])
            for x2 in range(len(eva_list_ts[i]['store_id'].unique())):
                o = np.zeros(bottom_level_number)
                o[x2::9] = 1
                l[i].append(o)
        elif i == 3:  # （2160，2160）
            # for l4
            l.append(np.eye(bottom_level_number))
        else:
            break
        i += 1
    S = np.concatenate((l[0], l[1], l[2], l[3]), axis=0)
    return S

In [49]:
S_Matrix = SMatrix(4,2160, eva_list)

In [50]:
np.shape(S_Matrix)

(2174, 2160)

In [229]:
######## G matrix for BU
g1 = np.zeros((2160,14))#（ bottom_level , except_bottom level）
g2 = np.eye(2160)# bottom level
G_bu = np.concatenate((g1,g2), axis=1)
G_bu

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [230]:
np.shape(G_bu)

(2160, 2174)

In [231]:
SG_bu = np.dot(S_Matrix, G_bu)

In [232]:
############# for all 3 reconciliation methods
y_base_frec = test_pred.reshape(int(len(y_test)/28),28)

In [233]:
y_coh_frec_bu = np.dot(SG_BU, y_base_frec)
y_coh_frec_bu

array([[2.87271809e+03, 2.83518813e+03, 2.82170303e+03, ...,
        3.47435950e+03, 3.66653762e+03, 3.59506159e+03],
       [1.31792136e+03, 1.29900247e+03, 1.26898251e+03, ...,
        1.49236265e+03, 1.59521869e+03, 1.62863936e+03],
       [6.68449743e+02, 6.80509193e+02, 6.97948392e+02, ...,
        8.83724379e+02, 9.29024450e+02, 9.01336129e+02],
       ...,
       [3.43297176e+00, 3.01815797e+00, 2.88633280e+00, ...,
        2.84748045e+00, 3.11658267e+00, 2.38970711e+00],
       [3.73699386e+00, 2.52582476e+00, 1.63335436e+00, ...,
        2.06502376e+00, 1.95800698e+00, 1.47116428e+00],
       [2.46367824e+00, 2.05463634e+00, 3.05996365e+00, ...,
        8.41853415e-01, 1.02323021e+00, 1.34062698e+00]])

In [234]:
# BU's MASE
re_bu = forecast_mase(y_coh_frec_bu.reshape(60872,1), y_test)
np.mean(re_bu)

0.005343179454768409

In [242]:
###########################TD
# G_Matrix_TD
def G_Matrix_TD(bottom_level, top_level, ts_length, col_bo, col_to, ts_amount):
    from typing import Optional
    from tqdm import tqdm
    prop_mean = []
    n = int(len(bottom_level[col_bo])/ts_length)
    for i in tqdm(range(n)):
        prop = []
        #prop = np.divide(np.array(bottom_level[col_bo][i*ts_length : (i+1)*ts_length]), np.array(top_level[col_to]))
        #, out=np.zeros_like(vector1), where=(vector2 != 0)
        with np.errstate(divide='ignore', invalid='ignore'):
            prop =  (bottom_level[col_bo].iloc[i*ts_length : (i+1)*ts_length].values/top_level[col_to].values)
        prop_mean.append(prop)
    prop_mean = np.nan_to_num(prop_mean)
    prop_mean = np.mean(prop_mean,axis = 1).reshape(-1,1)
    g2 = np.zeros((n, ts_amount-1))
    G = np.concatenate((prop_mean, g2), axis=1)
    return G

In [243]:
G_td = G_Matrix_TD(eva_list[3], eva_list[0], 1941, 'sales', 'sales', 2174)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2160/2160 [00:00<00:00, 42351.85it/s]


In [244]:
SG_td = np.dot(S_Matrix, G_td)
SG_td

array([[9.98969603e-01, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [4.65835429e-01, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.51465335e-01, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [1.04392435e-03, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [9.02260842e-04, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [8.24399064e-04, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [245]:
y_coh_frec_td = np.dot(SG_TD, y_base_frec)
y_coh_frec_td

array([[2.53619861e+03, 2.77715748e+03, 2.76623876e+03, ...,
        3.52296629e+03, 3.48289078e+03, 3.44206800e+03],
       [1.18266979e+03, 1.29503274e+03, 1.28994117e+03, ...,
        1.64281526e+03, 1.62412741e+03, 1.60509110e+03],
       [6.38423865e+02, 6.99079166e+02, 6.96330654e+02, ...,
        8.86817673e+02, 8.76729677e+02, 8.66453575e+02],
       ...,
       [2.65033038e+00, 2.90213266e+00, 2.89072259e+00, ...,
        3.68150369e+00, 3.63962474e+00, 3.59696490e+00],
       [2.29067300e+00, 2.50830499e+00, 2.49844330e+00, ...,
        3.18191316e+00, 3.14571730e+00, 3.10884652e+00],
       [2.09299638e+00, 2.29184754e+00, 2.28283687e+00, ...,
        2.90732581e+00, 2.87425352e+00, 2.84056455e+00]])

In [246]:
re_td = forecast_mase(y_coh_frec_td.reshape(60872,1), y_test)
print(np.mean(re_td))

0.005889916583258045


In [247]:
np.shape(S_Matrix)

(2174, 2160)

In [249]:
test_pred

array([2.53881460e+03, 2.78002201e+03, 2.76909202e+03, ...,
       8.41853415e-01, 1.02323021e+00, 1.34062698e+00])

In [253]:
y_test

,y_1DaysF
1912,2841.0
1913,2860.0
1914,2799.0
1915,3184.0
1916,3800.0
...,...
4192554,0.0
4192555,0.0
4192556,0.0
4192557,0.0


In [322]:
########  MinT‘s G matrix ############
### estimate Wh matrix at first # size (2174*2174)
def W_matrix_G(forecast, truth):
    residuals_ = []
    for i in range(int(len(forecast)/28)):
        res = np.mean(truth[i*28:(i+1)*28]-forecast[i*28:(i+1)*28])
        residuals_.append(res)
    W = np.diag(residuals_)
    return W, residuals_

In [323]:
W , residuals =W_matrix_G(test_pred, y_test.values)

In [324]:
residuals
np.shape(residuals)
np.shape(W)

(2174, 2174)

In [325]:
G_mint = np.linalg.inv(S_Matrix.T@np.linalg.inv(W)@S_Matrix)@S_Matrix.T@np.linalg.inv(W)

In [327]:
np.shape(G_mint)

(2160, 2174)

In [329]:
SG_mint = S_Matrix@ G_mint
SG_mint

array([[ 1.39467762e-01,  2.96038807e-01,  2.24310686e-01, ...,
         2.95284200e-01,  1.40666688e-01,  2.98164743e-01],
       [ 5.35015915e-02,  4.81488762e-01, -9.35368385e-02, ...,
        -8.10018608e-02, -1.25259970e-01, -9.28271006e-02],
       [ 3.38466710e-02, -7.80963304e-02,  4.03864190e-01, ...,
        -2.80528367e-02, -7.41395416e-02, -2.83572009e-02],
       ...,
       [ 1.03666932e-04, -1.57353706e-04, -6.52695809e-05, ...,
         9.96279357e-01, -5.72446629e-04, -6.87004324e-04],
       [-4.79658907e-05,  2.36339057e-04,  1.67542562e-04, ...,
         5.56001826e-04,  1.00450856e+00,  5.81130619e-04],
       [-1.98973675e-04,  3.42764679e-04,  1.25411308e-04, ...,
         1.30586646e-03,  1.13729162e-03,  1.00688763e+00]])

In [330]:
y_coh_frec_mint = SG_mint@y_base_frec
y_coh_frec_mint

array([[2.55790195e+03, 2.64310586e+03, 2.65508267e+03, ...,
        3.45419990e+03, 3.82591726e+03, 3.49061909e+03],
       [1.20144222e+03, 1.22825153e+03, 1.21466945e+03, ...,
        1.46648008e+03, 1.74573483e+03, 1.65020482e+03],
       [6.00509421e+02, 6.36086081e+02, 6.60236179e+02, ...,
        8.58845069e+02, 9.25763092e+02, 8.48829901e+02],
       ...,
       [3.04579832e+00, 2.80700003e+00, 2.75870140e+00, ...,
        2.89343063e+00, 2.93148874e+00, 2.01694533e+00],
       [3.98440311e+00, 2.80577049e+00, 1.78717284e+00, ...,
        1.93706592e+00, 1.70149784e+00, 1.72471699e+00],
       [2.41485787e+00, 2.02047598e+00, 3.12912381e+00, ...,
        2.66187976e-01, 3.72410398e-01, 1.26035999e+00]])

In [331]:
re_mint = forecast_mase(y_coh_frec_mint.reshape(60872,1), y_test)
print(np.mean(re_mint))

0.005244599671068256


In [335]:
print("lgbm's forecasts:", np.mean(mase_base))
print("lgbm with BU's forecasts:", np.mean(re_bu))
print("lgbm with TD's forecasts:", np.mean(re_td))
print("lgbm with MinT's forecasts:", np.mean(re_mint))

lgbm's forecasts: 0.00492575781142385
lgbm with BU's forecasts: 0.005343179454768409
lgbm with TD's forecasts: 0.005889916583258045
lgbm with MinT's forecasts: 0.005244599671068256
